In [1]:
import patchmatch as pm
import imageio

import numpy as np

/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'patchmatch' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
def patch_match(source_img, target_img):
    height, width, _ = source_img.shape
    offsets = np.empty((height, width, 2), dtype=np.float32)
    costs = np.empty((height, width), dtype=np.float32)

    PATCHMATCH_SIZE = 8
    MAX_OFFSET = 100
    MIN_OFFSET = 10
    pm.pm(source_img.astype(np.float32),
          target_img.astype(np.float32),
          offsets,
          costs,
          PATCHMATCH_SIZE,
          MIN_OFFSET,
          MAX_OFFSET)

    return offsets[:, :, ::-1]

In [3]:
imgs = []
img = imageio.imread('bike_a.png')
img = (img / 255).astype(np.float64)
imgs.append(img)
img = imageio.imread('bike_b.png')
img = (img / 255).astype(np.float64)
imgs.append(img)
# lib_offsets = patch_match(imgs[0], imgs[1])
# np.save('lib_offset.npy', lib_offsets)
lib_offsets = np.load('lib_offset.npy')

In [4]:
import os

import pyopencl as cl

os.environ['PYOPENCL_NO_CACHE'] = '1'
os.environ['PYOPENCL_NO_CACHE'] = '1'
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

filename = 'patchmatch.c'
PATCH_SIZE = 8


HEIGHT, WIDTH, _ = img.shape
ctx = cl.create_some_context(False)
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
program = cl.Program(ctx, open(filename).read()).build()

# NNF_HEIGHT = HEIGHT - (HEIGHT % PATCH_SIZE)
# NNF_WIDTH = WIDTH - (WIDTH % PATCH_SIZE)
NNF_HEIGHT = HEIGHT - PATCH_SIZE
NNF_WIDTH = WIDTH - PATCH_SIZE
nnf = np.empty((NNF_HEIGHT, NNF_WIDTH, 3))

In [ ]:
inputBuf = [cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=imgs[i]) for i in range(2)]
outputBuf = cl.Buffer(ctx, mf.WRITE_ONLY | mf.COPY_HOST_PTR, nnf.nbytes, hostbuf=nnf)

GLOBAL_SIZE = (NNF_HEIGHT, NNF_WIDTH)
program.randomfill(queue, GLOBAL_SIZE, None,
                                np.int32(PATCH_SIZE),  # patchHeight
                                np.int32(PATCH_SIZE),  # patchWidth
                                np.int32(HEIGHT),  # height
                                np.int32(WIDTH),  # width
                                inputBuf[0],
                                inputBuf[1],
                                outputBuf)


init_nnf = np.empty_like(nnf)
cl.enqueue_copy(queue, init_nnf, outputBuf)
nnf = np.copy(init_nnf)

In [ ]:
# mf = cl.mem_flags

for i in range(20):
    mf = cl.mem_flags
    inputBuf = [cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=imgs[i]) for i in range(2)]
    outputBuf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, nnf.nbytes, hostbuf=nnf)

    ITER = i + 1
    program.propagate(queue, GLOBAL_SIZE, None,
                                   np.int32(PATCH_SIZE),  # patchHeight
                                   np.int32(PATCH_SIZE),  # patchWidth
                                   np.int32(HEIGHT),  # height
                                   np.int32(WIDTH),  # width
                                   np.int32(ITER),
                                   inputBuf[0],
                                   inputBuf[1],
                                   outputBuf)


    output_nnf = np.empty_like(nnf)
    cl.enqueue_copy(queue, output_nnf, outputBuf)
    nnf = output_nnf.copy()

In [ ]:
tmp = nnf.copy()
tmp

In [ ]:
nnf.astype(np.int)

In [ ]:
grids = np.meshgrid(np.arange(NNF_WIDTH), np.arange(NNF_HEIGHT), indexing="xy")
cord = np.concatenate((grids[1][..., np.newaxis], grids[0][..., np.newaxis]), axis=2)

In [ ]:
a = nnf[:, :, :2] - cord
a - lib_offsets.astype(np.int)[:NNF_HEIGHT, :NNF_WIDTH]

In [ ]:
lib_offsets.astype(np.int)[:NNF_HEIGHT, :NNF_WIDTH]

In [ ]:
output_nnf.astype(np.int)

In [ ]:
nnf

In [ ]:
nnf[100, 200]

In [ ]:
lib_offsets[100, 210]

In [ ]:
imgs[0][100, 200]

In [ ]:
imgs[1][114, 207]

In [ ]:
imgs[1][115, 201]

In [ ]:
imgs[0].shape